In [76]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import pandas as pd
import gspread
import json

from sqlalchemy import create_engine


from gspread_dataframe import set_with_dataframe
from pipedrive.client import Client
# from main.apps.pat_bot.models import ZcomProspects
gc = gspread.service_account(filename="./credentials.json")

import environ
env = environ.Env()
database_url = env.str(
    var="DATABASE_URL", default="postgres://postgres:@postgres:5432/postgres"
)
engine = create_engine(database_url, echo=False)
client = Client(domain='https://codium2.pipedrive.com/')
client.set_api_token('c70f38803bb436089bce792681cd178e8459c86e')

In [77]:
def col_name_piped_act():
    return ['id', 'company_id', 'user_id', 'done', 'type', 'reference_type', 'reference_id',
      'conference_meeting_client', 'conference_meeting_url', 'due_date', 'due_time', 'duration',
      'busy_flag', 'add_time', 'marked_as_done_time', 'last_notification_time',
      'last_notification_user_id', 'notification_language_id', 'subject', 'public_description',
      'calendar_sync_include_context', 'location', 'org_id', 'person_id', 'deal_id', 'lead_id', 'lead_title',
      'active_flag', 'update_time', 'update_user_id', 'gcal_event_id', 'google_calendar_id',
      'google_calendar_etag', 'source_timezone', 'rec_rule', 'rec_rule_extension', 'rec_master_activity_id',
      'conference_meeting_id', 'note', 'created_by_user_id', 'location_subpremise', 'location_street_number',
      'location_route', 'location_sublocality', 'location_locality', 'location_admin_area_level_1',
      'location_admin_area_level_2', 'location_country', 'location_postal_code',
      'location_formatted_address', ]

In [107]:
def load_pipedrive_data():
    col = col_name_piped_act()

    # activities
    act_data = client.activities.get_all_activities(
        params={"limit": "100000"}
    )
    act_data = pd.DataFrame(act_data["data"])
    act_data = act_data[col]

    act_data.to_sql("etl_sale_activities", engine, if_exists="replace", index=False)

    # stages
    stages_data = client.stages.get_all_stages()
    stages_data = pd.DataFrame(stages_data["data"])

    stages_data.to_sql("etl_sale_stages", engine, if_exists="replace", index=False)

    # deals
    product_df = get_pipedrive_product()
    product_df = product_df[["product_id", "product_name", "deal_id"]]
    deals_data = client.deals.get_all_deals(params={"limit": "100000"})
    deals_data = pd.DataFrame(deals_data["data"])
    deals_data = deals_data[deals_data['pipeline_id'] == 10]

    deals_data["creator_user_id"] = list(
        map(lambda x: json.dumps(x), deals_data["creator_user_id"])
    )
    deals_data["person_id"] = list(
        map(lambda x: json.dumps(x), deals_data["person_id"])
    )
    deals_data["user_id"] = list(map(lambda x: json.dumps(x), deals_data["user_id"]))
    deals_data["org_id"] = list(map(lambda x: json.dumps(x), deals_data["org_id"]))

    ddf_re = deals_data.rename(columns={"id": "deal_id", 'd25eed5492b16b35a856444f2a9663c1c0261584' : 'sales_person',
                                       '322d77c1eb7f3a30eac9d35c874cace212959cec': 'One_time_fee', 
                                       '583b1978a81b12771c0ba83ae6fbe088d7edd11f': 'Monthly_fee'})
    ddf_re = get_detail_sale(ddf_re)
    ddf_join = ddf_re.join(product_df.set_index("deal_id"), on="deal_id")

    ddf_join.to_sql("etl_sale_deals", engine, if_exists="replace", index=False)

    # deal time range of stage
    deal_id = list(deals_data["id"])

    time_stage = get_pipedrive_stages_time(deal_id)

    time_stage.to_sql(
        "etl_sale_deals_time_of_stage_open", engine, if_exists="replace", index=False
    )


In [108]:
def get_pipedrive_product():
    product = client.products.get_all_products(params={"limit": "100000"})
    product = pd.DataFrame(product["data"])
    product_id = list(product["id"])

    product_list = list()
    for i in product_id:
        pdp = client.products.get_product_deal(str(i))
        if pdp["data"]:
            for j in pdp["data"]:
                product_list.append({"product_id": i, "deal_id": j["id"]})
    product_list = pd.DataFrame(product_list)
    product_re = product.rename(columns={"id": "product_id"})
    product_df = product_list.join(product_re.set_index("product_id"), on="product_id")
    product_df = product_df.rename(columns={"name": "product_name"})

    return product_df

In [109]:
def get_pipedrive_stages_time(data):
    get_deals_list = list()
    for i in data:
        deal_de = client.deals.get_deal(deal_id=int(i))
        get_deals_list.append(deal_de["data"])
    gd_pd = pd.DataFrame(get_deals_list)
    time_stay_stage = gd_pd[['stay_in_pipeline_stages', 'id', 'pipeline_id', 'status']]

    df = time_range_stage(time_stay_stage)

    ddf = time_stay_stage.merge(df, 'inner', on='id')
    ddf = ddf.rename(columns={'id': 'deal_id'})
    ddf = ddf.drop(columns=['stay_in_pipeline_stages'])

    return ddf

In [110]:
def time_range_stage(data):
    time_list2 = []
    stage_pipeline = {1: [1, 62, 2, 69, 5, 4, 35], 5: [24, 64, 25, 68, 26, 27, 65],
                      10: [55, 61, 80, 56, 79, 81, 67, 57, 58]}
    for idx, row in data.iterrows():
        time_range = row['stay_in_pipeline_stages']['times_in_stages']
        if row['status'] in ['won', 'lost', 'open']:
            if row['pipeline_id'] == 1:
                time_list2.append(cal_time(time_range, row, stage_pipeline[1]))

            elif row['pipeline_id'] == 5:
                time_list2.append(cal_time(time_range, row, stage_pipeline[5]))

            elif row['pipeline_id'] == 10:
                time_list2.append(cal_time(time_range, row, stage_pipeline[10]))

    tl_df = pd.DataFrame(time_list2)

    return tl_df

In [111]:
def cal_time(time_range, row, stage):
    time_dict = {}
    time_sum = 0
    sum_time_list = []

    for ind in stage:
        if str(ind) in time_range.keys():
            time_sum += time_range[str(ind)]
        sum_time_list.append(time_sum)
    time_dict['lead_to_meeting'] = sum_time_list[3]
    time_dict['meeting_to_Final'] = sum_time_list[6]
    time_dict['id'] = row['id']

    return time_dict

In [112]:
def get_detail_sale(data):
    data.reset_index(drop=True, inplace=True)
    sales_name = {37: 'Toon', 38: 'Jeff', 39 : 'Chompoo'}
    for i, row in data.iterrows():
        if row['sales_person']:
            word = list(map(int, row['sales_person'].split(',')))
            data.sales_person.iloc._setitem_with_indexer(i, sales_name[word[0]])
    return data

In [113]:
load_pipedrive_data()

<ipython-input-112-5372860ca877>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sales_person.iloc._setitem_with_indexer(i, sales_name[word[0]])


In [114]:
for i in range(33):
    print('จัด', end='')

จัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัดจัด